# Segmentation test

In [2]:
import cv2
import numpy as np
import time
from tensorflow.python.keras import Model
from tensorflow.python.keras.models import model_from_json
from IPython.display import clear_output

In [3]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
LABEL = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
         'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
         'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, 
         'del':26, 'nothing':27, 'space':28}

In [5]:
# Function to find angle between two vectors
def Angle(v1, v2):
    dot = np.dot(v1, v2)
    x_modulus = np.sqrt((v1 * v1).sum())
    y_modulus = np.sqrt((v2 * v2).sum())
    cos_angle = dot / x_modulus / y_modulus
    angle = np.degrees(np.arccos(cos_angle))
    return angle

# Function to find distance between two points in a list of lists
def FindDistance(A, B): 
    return np.sqrt(np.power((A[0][0] - B[0][0]), 2) + np.power((A[0][1] - B[0][1]), 2)) 

#Perform morphological transformations to filter out the background noise
def clean_background(frame):
    #Convert to HSV color space
    hsv_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #Create a binary image with where white will be skin colors and rest is black
    mask = cv2.inRange(hsv_frame,np.array([2,50,50]),np.array([15,255,255]))
    
    #Kernel matrices for morphological transformation    
    kernel_square  = np.ones((11,11),np.uint8)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    #Dilation increase skin color area
    #Erosion increase skin color area
    dilation  = cv2.dilate(mask,kernel_ellipse,iterations = 1)
    erosion   = cv2.erode(dilation,kernel_square,iterations = 1)    
    dilation2 = cv2.dilate(erosion,kernel_ellipse,iterations = 1)    
    filtered  = cv2.medianBlur(dilation2,5)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    dilation2 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation3 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    median    = cv2.medianBlur(dilation2,5)
    return cv2.threshold(median,127,255,0)[1]

In [51]:
#Open Camera object
cap = cv2.VideoCapture(0)

# Decrease frame size
# Note this may not work, only depends on camera support
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 200)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 200)
cap.set(cv2.CAP_PROP_FPS, 5)

# # Creating a window for HSV track bars
# cv2.namedWindow('HSV_TrackBar')

while(True):
    start_time = time.time()
    
    #Capture frame
    frame = cap.read()[1]

    threshold = clean_background(frame)
    
    #Find contours of the filtered frame
    _, contours, hierarchy = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    
    #Draw Contours
#     cv2.drawContours(frame, contours, -1, (122,122,0), 3)
    
    #Find Max contour area (Assume that hand is in the frame)
    max_area = 250
    ci = 0
 
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if(area > max_area):
            max_area = area
            ci = i  
    
    #Largest area contour  
    try:
        cnts = contours[ci]
    except IndexError:
        print("No contour found")
        break

    #Find convex hull
    hull = cv2.convexHull(cnts)
    
    #Find moments of the largest contour
    moments = cv2.moments(cnts)
    
    #Find convex defects
#     cv2.drawContours(frame, [hull], -1, (255, 255, 255), 2)
    
    #Central mass of first order moments
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00']) # cx = M10/M00
        cy = int(moments['m01'] / moments['m00']) # cy = M01/M00
    centerMass=(cx, cy)    
    
    #Print bounding rectangle
    x, y, w, h = cv2.boundingRect(cnts)
    img = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    box = img[y:y+h, x:x+w]
    box = cv2.cvtColor(box, cv2.COLOR_BGR2GRAY)
#     print(box.shape)
    box = cv2.resize(box, (200, 200), interpolation=cv2.INTER_CUBIC) 
    cv2.imshow('Box', box)
    # Show final image
    pred = np.empty((1, 200, 200))
    pred[0] = box
    probabilities = loaded_model.predict(pred / np.max(pred))
    predictions = np.argmax(probabilities, axis=-1)
    letter = list(LABEL.keys())[predictions[0]]
    cv2.putText(frame, letter, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)     
    cv2.imshow('Dilation', frame)
    
    #Print execution time
#     print("execution time per frame:\t{:.2f} [ms]".format((time.time()-start_time) * 1000))
    
    #close the output video by pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

No contour found


-1

In [39]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1